In [ ]:
%matplotlib notebook
%pylab

In [ ]:
# user defined parameters
u = 2   # velocity
Tf = 0.025  # final time
ax = -1; bx = 1
sigma = 1/8
#qinit = lambda x: np.exp(-(x/(2*sigma))**2)
qinit = lambda x : sin(2*pi*x)

# function f(q(x,t)) = u*q(x,t)

fe = lambda q: u*q


In [ ]:
N = 3 # order of the legendre polynomial
Q = N+1
a = 0.01  # CFL number

In [ ]:
def Lobatto_deriv(Q, x):
    F = [] # array containing the polynomials
    dF = []
    ddF = []


    f0 = 1; F.append(f0)  # first polynomial
    df0 = 0; dF.append(df0)
    f1 = x; F.append(f1)  # second polynomial
    df1 = 1; dF.append(df1)
    ddF = [0,0]

    B = []
    dB = []


    for i in range(2, Q+1):
        fi = ((2*i-1)/i)*x*F[i-1] - ((i-1)/i)*F[i-2]  # iteration of the polynomials
        dfi = i*F[i-1] + x*dF[i-1]                    # first derivative
        F.append(fi)
        dF.append(dfi)
        ddfi = (i+1.0)*dF[i-1] + x*ddF[i-1]           # second derivative
        ddF.append(ddfi)

        bi = (1.0-x**2)*dF[i-1]                       # lobatto polynomial
        dbi = -2.0*x*dF[i-1] + (1.0-x**2)*ddF[i-1]    # derivative of lobatto polynomial
        B.append(bi)
        dB.append(dbi)
        
    return B[-1], dB[-1]

In [ ]:
def Legendre_deriv(Q, x):
    L = [] # array containing the polynomials
    dL = []


    f0 = 1; L.append(f0)  # first polynomial
    df0 = 0; dL.append(df0)
    f1 = x; L.append(f1)  # second polynomial
    df1 = 1; dL.append(df1)
    

    for i in range(2, Q+1):
        fi = ((2*i-1)/i)*x*L[i-1] - ((i-1)/i)*L[i-2]  # iteration of the polynomials
        dfi = i*L[i-1] + x*dL[i-1]
        L.append(fi)
        dL.append(dfi)
        
    return L[-1], dL[-1]

In [ ]:
def Lobatto_p(Q):
    X = []  # Array that contains legendre points
    K = 100  # Order of approximation of Newton method
    e = 10**(-20) #error
    for i in range(0, Q+1):
        xi0 = cos(((2*i+1)/(2*Q+2))*pi)   # Chebchev points
        

        xik = xi0

        for k in range(0, K):
            out1, out2 = Lobatto_deriv(Q+1, xik)
            xikk = xik - out1/out2   # approximation of the solution using Newton

            if abs(xikk-xik) < e:

                break

            xik = xikk

        X.append(xikk)
    return X

In [ ]:
def LagrangeBasis(N, i, xl, Xr):
    
    L = 1
    dL = 0
        
    for j in range(N+1):
            
        prod = 1
        
        if (j != i):
            L = L*(xl-Xr[j])/(Xr[i]-Xr[j])
                
            for k in range(N+1):
                if (k!=i  and k!=j):
                    prod = prod*(xl-Xr[k])/(Xr[i]-Xr[k])
        
            dL = dL+prod/(Xr[i]-Xr[j])
    return L, dL

## Array intma

In [ ]:
def intma(e, N):
    t = (e-1)*N
    r = N*e
    intmm = []
    for s in range(t, r+1):
        intmm.append(s)
    
    return array(intmm)


## Weight values

In [ ]:
#funtion that compute weight values based on quadrature rule
def weight(Q):
    xi = Lobatto_p(Q)
    w = zeros(Q+1)
    for i in range(Q+1):
        out1, out2 = Legendre_deriv(Q, xi[i])
        w[i] = 2/(Q*(Q+1)*(out1)**2)
        
    return w 

#weight(Q)

## Element mass matrix

In [ ]:
def Element_matrix(N,Q):
    Me = zeros((N+1, N+1))       # initialisation of the matrix
    Xr = Lobatto_p(N)               # roots
    Xi = Lobatto_p(Q)               # Lobatto points
    w = weight(Q)                   # weight values

    for i in range(N+1):

        for j in range(N+1):
            for k in range(Q+1):
                xi, a1 = LagrangeBasis(N, i, Xi[k], Xr)
                xj, a2 = LagrangeBasis(N, j, Xi[k], Xr)
                Me[i,j] = Me[i,j]+ w[k]*xi*xj



    Me = (1/2)*Me

    return Me

#Element_matrix(N,Q)

## Global mass matrix

In [ ]:
def GMM(Ne, intma, N, Q):
    
    Np = Ne*N+1
    
    w = weight(Q)
    Me = Element_matrix(N,Q)
    M = zeros((Np, Np))
    
    for e in range(1,Ne+1):
        for j in range(N+1):
            
            J = intma(e, N)[j]
            
            for i in range(N+1):
                
                I = intma(e, N)[i]
                
                M[I,J] = M[I,J] + ((bx-ax)/Ne)*Me[i,j]
    return M

#GMM(Ne, intma, N, Q)

## Differential element matrix

In [ ]:
def Element_Diff_matrix(N):
    De = zeros((N+1, N+1))
    Xi = Lobatto_p(N)
    w = weight(N)

    for i in range(N+1):

        for j in range(N+1):
            for k in range(N+1):
                xi, a1 = LagrangeBasis(N, i, Xi[k], Xi)
                a2, xj = LagrangeBasis(N, j, Xi[k], Xi)
                De[i,j] = De[i,j]- w[k]*xi*xj

    return De

De = Element_Diff_matrix(N)

# Function for that compute residual global vector

In [ ]:
# function that compute global residual vector

def Resi(Ne, q, N,Miv):
    
    Np = Ne*N+1
    
    fe = lambda q: u*q
    
    R = zeros(Np)                 # global residual vector
    
    for e in range(1, Ne+1):         # element loop

        # discretizing into element 

        Ie  = intma(e, N)
        
        # discretizing into element 
        
        qe = q[Ie]
               
        # residual for each element

        Re = zeros(N+1)

        for i in range(N+1):
        
            Re[i] = - dot(De[i],fe(qe))
            
            # compuataion of global residual vector

            I = Ie[i] #intma(e,N)[i]            

            R[I] = R[I] + Re[i]
    
    # reinitialisation of the global residual vector using inverse mass matrix
    
    GR = zeros(Np) 
    
    for I in range(Np):
        
        GR[I] = dot(Miv[I],R)
        
    return GR

# Solver for 1D wave equation: Time integration (RK4)

In [ ]:
def Solver_1DW(N,Ne,M, x,t,dt):
    
    
    #q0 = qinit(x)         # defined initial condition
    q0 = array([qinit(l) for l in x])
    # Boundary conditions

    q0[0] = qinit(ax) 
    q0[-1] = qinit(bx) 

    
    #inverse of global mass matrix
    
    MM = GMM(Ne, intma, N, Q)
    
    Miv = linalg.inv(MM)     

    # computation of the solution of 1D wave equation
    qn = q0     

    for n in range(M):                   # time loop

        K1 = Resi(Ne, qn,N, Miv)

        # soultion for the wave equation at time n+1

        qh = qn + (dt/2)*K1

        K2 = Resi(Ne,qh, N,Miv)
        
        Ph = qn + (dt/2)*K2
        
        K3 = Resi(Ne, Ph, N, Miv)
        
        P = qn + dt*K3
        
        K4 = Resi(Ne, P, N, Miv)
        

        qn1 = qn + (dt/6)*(K1+2*K2+2*K3+K4) 

        qn1[0] = qinit(ax-u*t[n+1])
        qn1[-1] = qinit(bx-u*t[n+1])
        
        qn = qn1
    
    return qn1

In [ ]:
# list of the number of elements

NE = array([20,40,80,160])
l = len(NE)

# array that contains the error for each iteration
Error = zeros(l)

for ne in range(l):
    
    Ne = NE[ne]
    Np = N*Ne+1
    
    # space stuff
    #x = np.linspace(-1, 1, Np)
    
    x = zeros(Np)
    xel = linspace(-1,1,Ne+1)
    xe2 = array(Lobatto_p(N))
    xe = xe2[::-1]
    for e in range(1,Ne+1):
        ae=xel[e-1] ; be=xel[e]
       
        xsi=((be-ae)/2)*(xe-1) + be
        #print('a=',a)
        #print('b=',b)
        #print('xsi=',xsi)
        for i in range(0,N+1):
            ip=intma(e,N)[i]
            #print('ip=',ip)
            x[ip]=xsi[i]
    #x = np.linspace(-1, 1, Np)
    
    dx = (bx-ax)/(Np-1)   # for second order

    #time stuff

    fixed_dt= True
    if (fixed_dt):
        dx0 = (bx-ax)/(NE[0]*N)
        dtest = a*dx0/np.abs(u)
        M0 = int(Tf//dtest)+1    # Number of time steps
        dt0 = Tf/M0
    
        f = log2(dx0/dx)
        dt = dt0/2**f
        M = int(M0*2**f)
        print('M = {:d}'.format(M))
    else:        
        dtest = a*dx/np.abs(u)
        M = Tf//dtest+1    # Number of time steps
        M0 = int(M)
        dt = Tf/M

    t = linspace(0, Tf, M+1)


    # approximate solution of the wave equation
    q = Solver_1DW(N,Ne,M,x,t,dt)
    
    # exact solution
    q_exact = qinit(x-u*t[M])
    #error
    error = abs(q-q_exact)
    Error[ne] = max(error)
    #error = sqrt(sum((q-q_exact)**2)/sum(q_exact**2))
    #Error[ne] = error
    #plot of the computed and exact solutions
    
    figure(ne)
    plot(x, q, label = 'Computed: Ne = {}'.format(Ne))
    plot(x, q_exact, '--', label = 'Exact: Ne = {}'.format(Ne))
    
    xlabel('NE')
    ylabel('q')
    legend()
    grid() 
    
    

In [ ]:
figure(7)
loglog(NE, Error,'k.',markersize=15)
show()

In [ ]:
Np = NE*N+1
p = polyfit(log(NE), log(Error),1)
p

In [ ]:
Error

In [ ]:
#error

In [ ]:
#x = log(NE[1]) - log(NE[0])
#y = log(Error[1]) - log(Error[0])

#slope = y/x
#slope

In [ ]:
Np